In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#from sklearn import preprocessing as pp
#from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import re
import time
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

#import torchvision.transforms as transforms
import torch.optim as optimizers
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler, BatchSampler

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def set_seed(seed: int = 123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(123)

In [4]:
all_data = pd.read_csv('../input/intern-compe4/all_for_nn.csv')
train_set = all_data[all_data["data_type"] == "train"]

In [5]:
target_cols = ["state"]
feature_cols = ['duration', 'goal_min',
                '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24',
       'number_of_figure', 'number_of_paragraph',
       'length_of_text', 'country_4',
       'country_16', 'country_21', 'country_9', 'country_3', 'country_7',
       'country_8', 'country_5', 'country_1', 'country_12', 'country_15',
       'country_18', 'country_17', 'country_6', 'country_10', 'country_20',
       'country_2', 'country_19', 'country_11', 'country_13', 'country_0',
       'country_14',
        'category1_12', 'category1_5', 'category1_7', 'category1_13',
       'category1_11', 'category1_0', 'category1_6', 'category1_3',
       'category1_4', 'category1_8', 'category1_9', 'category1_10',
       'category1_1', 'category1_14', 'category1_2']

In [6]:
class CFDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = csv_file
        self.transform = transform
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)
        try:
            target = self.csv_file[self.target_cols].iloc[idx]
            target = torch.tensor(target)
            # train_step
        except:
            target = self.csv_file["id"].iloc[idx]
            # test_step
        
        if self.transform:
            features = self.transform(features)
            
        return features, target

In [7]:
class CFDataModule(pl.LightningDataModule):
    def __init__(self, train, validation, transform, split_rate, batch_size, num_workers):
        super().__init__()
        self.csv_file = train
        self.transform = transform
        self.split_rate = split_rate
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.validation = validation
        

    def setup(self, stage=None):
        self.train_dataset = CFDataset(csv_file=self.csv_file, transform=self.transform)
        self.val_dataset = CFDataset(csv_file=self.validation, transform=self.transform)
            
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)

In [8]:
class CFModule(pl.LightningModule):
    def __init__(self, input_dim, hidden_sizes, out_dim, dropout):
        super(CFModule, self).__init__()
        
        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.utils.weight_norm(nn.Linear(input_dim, hidden_sizes[0]))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_sizes[0])
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_sizes[0], hidden_sizes[1]))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_sizes[1])
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_sizes[1], hidden_sizes[2]))

        self.layers = nn.Linear(hidden_sizes[2], out_dim)
            
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout(x)
        x = F.relu(self.dense3(x))
        
        x = self.layers(x)
        
        return torch.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("vall_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.001, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        #pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(input=pred, target=t)
    
    
    def metric(self, pred, t):
        t = t.to('cpu')
        try:
            pred = pred.detach().numpy()
        except:
            pred = pred.to('cpu')
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [9]:
def add_cols_kfold(n_folds, train, seed):
    cv = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    for fold, (t_idx, v_idx) in enumerate(cv.split(train)):
        train.loc[v_idx, 'kfold'] = int(fold)
        
    return train

In [10]:
def main():
    # trainer config
    epochs = 100
    output_path = './'
    
    # data module config
    #train_set
    transform = None
    split_rate = 0.8
    batch_size = 256*8
    num_workers = 4
    
    # model config
    input_dim = len(feature_cols)
    hidden_sizes = [120, 30, 70]
    dropout = 0.3
    out_dim = 1
    
    # early stopping config
    patience = 5
    
    # kfold config
    n_folds = 5
    seed = 123
    
    # prepare data for with kfold
    train = add_cols_kfold(n_folds=n_folds,
                            train=train_set,
                            seed=seed)
    
    for fold in train.kfold.unique():
        train_fold = train[train["kfold"] != fold]
        val_fold = train[train["kfold"] == fold]
    
        cf = CFDataModule(train_fold, val_fold, transform, split_rate, batch_size, num_workers)
        model = CFModule(input_dim=input_dim, hidden_sizes=hidden_sizes, out_dim=out_dim, dropout=dropout)

        early_stopping = EarlyStopping('val_loss', patience=patience, verbose=True)
        trainer = Trainer(
                max_epochs=epochs,
                weights_save_path=output_path,
                gpus = 1 if torch.cuda.is_available() else None,
                callbacks=[early_stopping]
                #accumulate_grad_batches=1
                # use_amp=False,
            )

        trainer.fit(model, cf)
        torch.cuda.empty_cache()
        # TO DO: use model.apply(weights_init) instead of torch.cuda.empty_cache()

In [11]:
if __name__ == "__main__":
    main()

EarlyStopping mode set to min for monitoring val_loss.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type        | Params
--------------------------------------------
0 | batch_norm1 | BatchNorm1d | 134   
1 | dropout     | Dropout     | 0     
2 | dense1      | Linear      | 8.3 K 
3 | batch_norm2 | BatchNorm1d | 240   
4 | dense2      | Linear      | 3.7 K 
5 | batch_norm3 | BatchNorm1d | 60    
6 | dense3      | Linear      | 2.2 K 
7 | layers      | Linear      | 71    


val_loss: 9.559722900390625, val_acc: 0.0


val_loss: 0.6891050338745117, val_acc: 0.06278855032317636


val_loss: 0.6812728643417358, val_acc: 0.5048543689320389


val_loss: 0.6676787734031677, val_acc: 0.6956942428640541


...

val_loss: 0.4629395604133606, val_acc: 0.7732201791607732


val_loss: 0.46292805671691895, val_acc: 0.7741631305987742


val_loss: 0.46295297145843506, val_acc: 0.7727916863486065


val_loss: 0.46302536129951477, val_acc: 0.774102079395085


val_loss: 0.4628811180591583, val_acc: 0.774560987185572

